# Jupyter Notebook Code for ArUco Marker Detection and Poster Overlay

# Import Libraries

In [1]:
import cv2
import numpy as np
import sys  
import os

# Section 1: Setup Directories and ArUco Detector

# Define directories

In [2]:
input_directory = "room_with_aruco_marker"
output_directory_markers = "aruco_markers_detection"
output_directory_overlay = "poster_overlayed"

# Create output directories if not exist
os.makedirs(output_directory_markers, exist_ok=True)
os.makedirs(output_directory_overlay, exist_ok=True)

In [3]:
# Get list of image file paths
image_files = [os.path.join(input_directory, f) for f in os.listdir(input_directory) if f.endswith(('.jpg', '.png'))]

# Define ArUco dictionary and parameters
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

# Create ArUco detector
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

# Section 2: Detect ArUco Markers
for image_path in image_files:
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not read image from '{image_path}'.")
        continue

    corners, ids, rejected = detector.detectMarkers(image)

    if len(corners) > 0:
        for i in range(len(corners)):
            corners_int = np.int32(corners[i][0])
            cv2.polylines(image, [corners_int], True, (0, 255, 0), thickness=5)
            cx = int(np.mean(corners_int[:, 0]))
            cy = int(np.mean(corners_int[:, 1]))
            cv2.putText(image, f"ArUco Marker Id: {ids[i][0]}", (200, 300), 
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 8)

        output_path = os.path.join(output_directory_markers, os.path.basename(image_path))
        cv2.imwrite(output_path, image)
        print(f"Markers detected and saved to '{output_path}'.")
    else:
        print(f"No ArUco markers detected in '{image_path}'.")


Markers detected and saved to 'aruco_markers_detection\20221115_113319.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113328.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113340.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113346.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113356.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113401.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113412.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113424.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113437.jpg'.
Markers detected and saved to 'aruco_markers_detection\20221115_113440.jpg'.
No ArUco markers detected in 'room_with_aruco_marker\20221115_113635.jpg'.


# Section 3: Poster Overlay Function

# Define a function to calculate the transformation matrix

In [4]:
# Define the function to calculate the transformation matrix
def calculate_transform_matrix(marker_corner, poster):
    e = marker_corner[0][0]
    f = marker_corner[0][1]
    g = marker_corner[0][2]
    h = marker_corner[0][3]
    
    height, width, _ = poster.shape
    center_x, center_y = width / 2, height / 2
    
    small_width = width / 9
    small_height = height / 6
    a = (center_x - small_width / 2, center_y - small_height / 2)
    b = (center_x + small_width / 2, center_y - small_height / 2)
    c = (center_x + small_width / 2, center_y + small_height / 2)
    d = (center_x - small_width / 2, center_y + small_height / 2)
    
    input_corners = np.float32([a, b, c, d])
    output_corners = np.float32([e, f, g, h])
    return cv2.getPerspectiveTransform(input_corners, output_corners)

# Section 4: Overlay Poster on Detected Markers

In [5]:
# Load the poster image
poster_path = "poster.png"
poster = cv2.imread(poster_path)

if poster is None:
    print(f"Error: Could not read poster image from '{poster_path}'.")
    sys.exit(1)  # Exit the script if the poster is not loaded

# Process each image in the directory
for image_path in image_files:
    # Read the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not read image from '{image_path}'.")
        continue

    # Detect ArUco markers
    corners, ids, _ = detector.detectMarkers(image)

    if len(corners) > 0:
        for i, marker_corner in enumerate(corners):
            # Calculate the transformation matrix
            M = calculate_transform_matrix(marker_corner, poster)

            # Apply perspective transformation to the poster
            transformed_poster = cv2.warpPerspective(
                poster, M,
                (image.shape[1], image.shape[0]),
                flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 1, 0)
            )

            # Create a mask for the transformed poster area
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
            cv2.fillPoly(mask, [marker_corner.astype(int)], 255)

            # Invert the mask to exclude the poster area from the original image
            inverted_mask = cv2.bitwise_not(mask)
            classroom_without_poster = cv2.bitwise_and(image, image, mask=inverted_mask)

            # Blend the transformed poster with the classroom image
            alpha = 1
            blended_image = cv2.addWeighted(transformed_poster, alpha, classroom_without_poster, 0, 1)
            
            # Create a copy of the classroom image
            classroom_copy = image.copy()
            
            # Integrate the blended image into the copy of the classroom image
            blended_image_gray = cv2.cvtColor(blended_image, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(blended_image_gray, 1, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)
            # Use the original image directly for the background
            classroom_without_poster = cv2.bitwise_and(image, image, mask=mask_inv)

            # Combine the classroom background and the transformed poster
            overlay_result = cv2.add(classroom_without_poster, blended_image)
            
            # Save the output image
            output_image_path = os.path.join(
                output_directory_overlay,
                f"result_image_{os.path.basename(image_path).split('.')[0]}_{i}.jpg"
            )
            cv2.imwrite(output_image_path, overlay_result)
            print(f"Overlay complete for marker {i + 1} in image '{os.path.basename(image_path)}' and saved to '{output_image_path}'.")
    else:
        print(f"No ArUco markers detected in '{image_path}'.")


Overlay complete for marker 1 in image '20221115_113319.jpg' and saved to 'poster_overlayed\result_image_20221115_113319_0.jpg'.
Overlay complete for marker 1 in image '20221115_113328.jpg' and saved to 'poster_overlayed\result_image_20221115_113328_0.jpg'.
Overlay complete for marker 1 in image '20221115_113340.jpg' and saved to 'poster_overlayed\result_image_20221115_113340_0.jpg'.
Overlay complete for marker 1 in image '20221115_113346.jpg' and saved to 'poster_overlayed\result_image_20221115_113346_0.jpg'.
Overlay complete for marker 1 in image '20221115_113356.jpg' and saved to 'poster_overlayed\result_image_20221115_113356_0.jpg'.
Overlay complete for marker 1 in image '20221115_113401.jpg' and saved to 'poster_overlayed\result_image_20221115_113401_0.jpg'.
Overlay complete for marker 1 in image '20221115_113412.jpg' and saved to 'poster_overlayed\result_image_20221115_113412_0.jpg'.
Overlay complete for marker 1 in image '20221115_113424.jpg' and saved to 'poster_overlayed\resul